# Lecture 7:LLM as a Judge



In [1]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_csv_agent

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["GOOGLE_API_KEY"] = os.getenv("GEMINI_API_KEY")

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest")
from datetime import datetime



In [5]:
agent = create_csv_agent(
    llm,
    "titanic.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True
)

In [6]:
agent.run("how many rows are there?")



C:\Users\aditya\AppData\Local\Temp\ipykernel_11400\779840898.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("how many rows are there?")




> Entering new AgentExecutor chain...
Thought: I can use the `.shape` attribute of the pandas DataFrame to get the number of rows and columns.  The first element of the tuple returned by `.shape` will be the number of rows.

Action: python_repl_ast
Action Input: 
```python
print(df.shape[0])
```891
I now know the final answer.

Final Answer: 891

> Finished chain.


'891'

In [15]:
# LLM as a Judge
data_base = []
message_counter = 0

while True:
    user_input = input("Enter your question: ")
    message_counter += 1
    user_message = f"user_message_{message_counter}" + " " + user_input
    data_base.append(user_message)

    if user_input == "end":
        break

    result = agent.run(user_input)
    agent_message = f"agent_message_{message_counter}" + " " + result
    data_base.append(agent_message)

    print(result)

llm_as_judge_prompt = f"""
You are a judge .You will output if the conversation is helpful or not based on the conversation and user_experience.
return "helpful" or "not helpful" only no explanation needed.

Conversation:
{data_base}
"""
print(data_base)
verdict=llm.invoke(llm_as_judge_prompt)
print(verdict.content)




> Entering new AgentExecutor chain...
Thought:The question is not related to the provided dataframe.  I need more information or a question related to the dataframe to proceed.

Final Answer: Hi

> Finished chain.
Hi


> Entering new AgentExecutor chain...
Thought: I need to count the number of passengers who did not survive.  This is indicated by a 'Survived' value of 0.
Action: python_repl_ast
Action Input: 
```python
df['Survived'].value_counts()
```1    3
0    2
Name: Survived, dtype: int64The output shows that there are 2 passengers with a 'Survived' value of 0, meaning they did not survive.
Thought: I now know the final answer
Final Answer: 2

> Finished chain.
2
['user_message_1 Hi', 'agent_message_1 Hi', 'user_message_2 How many people sinked in?', 'agent_message_2 2', 'user_message_3 end']
not helpful
